In [1]:
import json
import os.path

with open("../data/ComTQA/annotation.json") as f:
    data = json.load(f)

In [2]:
len(data)

9070

In [3]:
for item in data[:10]:
    print(item)

{'image_name': 'PMC1079905_table_1.jpg', 'question': 'What is the third function of the vasa vasorum?', 'answer': 'Substrates of native LDL-cholesterol and modified LDL-cholesterol \n Substrates of phospholipids from systemic circulating cells', 'dataset': 'PubTab1M'}
{'image_name': 'PMC1164435_table_2.jpg', 'question': 'How many rows are there in the table?', 'answer': '38', 'dataset': 'PubTab1M'}
{'image_name': 'PMC1083419_table_2.jpg', 'question': 'What is the median time off work for a 1-year-old child with a severe allergic reaction?', 'answer': '0', 'dataset': 'PubTab1M'}
{'question': 'What was the end-of-year ratio of fixed-to-floating debt in 2013?', 'answer': '75/25', 'dataset': 'FinTabNet', 'table_id': '23040'}
{'image_name': 'PMC1156872_table_2.jpg', 'question': 'What is the incidence of dysplasia in the group treated with AOM/DSS and 0.05% Befibrate?', 'answer': '80%', 'dataset': 'PubTab1M'}
{'image_name': 'PMC1065112_table_0.jpg', 'question': 'What was the least common com

In [5]:
cnt = 0
for item in data:
    dataset = item["dataset"]
    if dataset == 'PubTab1M':
        pass
    elif dataset == 'FinTabNet':
        cnt += 1
    else:
        print(item)
cnt

2838

In [10]:
import jsonlines

with open("../data/ComTQA/fintabnet/FinTabNet_1.0.0_table_test.jsonl") as f:
    data = jsonlines.Reader(f)
    data = list(data)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3e26267910>>
Traceback (most recent call last):
  File "/root/anaconda3/envs/llava/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [11]:
len(data)

7085

In [12]:
from pprint import pprint

pprint(data[0])

{'bbox': [115.9402, 602.0382000000001, 484.045024, 707.40584],
 'filename': 'HAL/2009/page_77.pdf',
 'html': {'cells': [{'tokens': []},
                    {'bbox': [377.35, 694.31, 430.91, 704.28],
                     'tokens': ['D',
                                'e',
                                'c',
                                'e',
                                'm',
                                'b',
                                'e',
                                'r',
                                ' ',
                                '3',
                                '1']},
                    {'bbox': [115.94, 680.75, 190.3, 690.72],
                     'tokens': ['M',
                                'i',
                                'l',
                                'l',
                                'i',
                                'o',
                                'n',
                                's',
                                ' 

In [8]:
import json
import fitz  # PyMuPDF
import os
from PIL import Image

def load_annotations(jsonl_file):
    annotations = []
    with open(jsonl_file, 'r') as f:
        for line in f:
            annotations.append(json.loads(line))
    return annotations


def crop_table_from_pdf(pdf_path, table_annotations, output_folder):
    for i, annotation in enumerate(table_annotations):
        filepath = os.path.join(pdf_path, annotation['filename'])
        doc = fitz.open(filepath)
        print(filepath)
        print(annotation['filename'])
        table_id = annotation['table_id']
        page_index = table_id - 1  # Assuming table_id is 1-based and corresponds to page number
        page = doc[0]
        bbox = annotation["bbox"]
        x0 = bbox[0]
        y0 = bbox[1]
        x1 = bbox[2]
        y1 = bbox[3]
        pix = page.get_pixmap(clip=fitz.Rect(bbox))
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img.save(f"./table_{table_id}.png")
        # cells = annotation['html']['cells']
        # if cells:
        #     x0, y0, x1, y1 = float('inf'), float('inf'), float('-inf'), float('-inf')
        #     for cell in cells:
        #         bbox = cell['bbox']
        #         x0 = min(x0, bbox[0])
        #         y0 = min(y0, bbox[1])
        #         x1 = max(x1, bbox[2])
        #         y1 = max(y1, bbox[3])
        #     
        #     # Crop the table image
        #     pix = page.get_pixmap(clip=fitz.Rect(x0, y0, x1, y1))
        #     img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        #     img.save(f"{output_folder}/table_{table_id}.png")
        break

    doc.close()


# Example usage
jsonl_file = '../data/ComTQA/fintabnet/FinTabNet_1.0.0_cell_test.jsonl'

pdf_path = '../data/ComTQA/fintabnet/pdf'
output_folder = 'cropped_tables'

annotations = load_annotations(jsonl_file)
crop_table_from_pdf(pdf_path, annotations, output_folder)

../data/ComTQA/fintabnet/pdf/HAL/2015/page_43.pdf
HAL/2015/page_43.pdf
